In [1]:
import gym
from gym.spaces import Discrete, Box
import numpy as np
import random
import itertools
import matplotlib.pyplot as plt
import cv2
import io
import base64
from IPython import display
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.animation as animation

In [2]:
class snake(gym.Env):
    
    def __init__(self):
        self.action_space = Discrete(4)
        self.videos_images = []
        self.snake_max_len = 100
        self.snake_starting_length = 3
        self.snake_length = 3
        self.rows = 10
        self.columns = 10
        self.moves = [(1, 0), (0, 1), (-1, 0), (0, -1)] # 0 is down, 1 is right, 2 is up, 3 is left
        self.default_board_environment = np.zeros((self.rows, self.columns))
        self.board_state = self.default_board_environment
        self.head_location = 0
        self.apple_location = 0
        self.videos = [('snake.mp4', 'snake.meta.json')]
    
    def render_video(self):
        lists = board.tolist()
        for n in range(len(lists)):
            lister = lists[n]
            for i in range(len(lister)):
                item = lister[i]
                done = False
                if item == 0:
                    lister[i] = [0, 0, 0]
                    done = True
                if item == 1000:
                    lister[i] = [255, 0, 0]
                    done = True
                if item == 100:
                    lister[i] = [0, 0, 244]
                if done == False:
                    lister[i] = [192, 192, 192]
            lists[n] = lister
        self.videos_images.append(lists)

    def write_video(self):
        fig, ax = plt.subplots()
        ims = []
        for i in range(len(self.videos_images)):
            im = ax.imshow(self.videos_images[i], animated=True)
            if i == 0:
                ax.imshow(self.videos_images[i])
            ims.append([im])
        ani = animation.ArtistAnimation(fig, ims, interval=50, blit=True,
                                        repeat_delay=1000)
        ani.save('snake.mp4')
        plt.close()
        

    def step(self, action):
        done = False
        #Get Move
        move = self.moves[action]
        #Make Move
        snake_loc = self.snake_location
        reward = -0.05
        cont = True
        moving = True
        new_length = False
        if (snake_loc[0]+move[0]) >= self.rows:
            cont = False
        elif (snake_loc[0] + move[0]) < 0:
            cont = False
        if (snake_loc[1]+move[1]) >= self.columns:
            cont = False
        elif (snake_loc[1]+move[1]) < 0:
            cont = False
        if cont == True:
            new_space = self.board_state[(snake_loc[0]+move[0]), (snake_loc[1]+move[1])]
            if new_space == 1000:
                done = True
                reward = 1
                new_length = True
            if done == False:
                if new_space != 0:
                    if new_space != (self.snake_max_len - self.snake_length):
                        done = True
                        reward = -1
        else:
            moving = False
            done = True
            reward = -1
        if moving == True:
            moved_snake = False
            self.board_state[(snake_loc[0]+move[0]), (snake_loc[1]+move[1])] = self.snake_max_len
            count = self.snake_length
            while moved_snake == False:
                if (self.snake_max_len - count) <= (self.snake_max_len - self.snake_length):
                    if new_length == True:
                        find = np.where(self.board_state == (self.snake_max_len-count))
                        self.board_state[find[0], find[1]] -= 1
                    else:
                        find = np.where(self.board_state == (self.snake_max_len-count))
                        self.board_state[find[0], find[1]] = 0
                else:
                    find = np.where(self.board_state == (self.snake_max_len-count))
                    self.board_state[find[0], find[1]] -= 1
                if count == 1:
                    moved_snake = True
                count -= 1
            self.board_state[(snake_loc[0]), (snake_loc[1])] -=1
            self.snake_location = [(snake_loc[0]+move[0]), (snake_loc[1]+move[1])]
        self.render_video()
        if done == True:
            self.write_video()
        return self.board_state, reward, done
    
    def reset(self):
        rand_row = random.randint(0, (self.rows-1))
        rand_col = random.randint(0, (self.columns-1))
        #Set board space and apple
        self.board_state = self.default_board_environment
        self.board_state[rand_row, rand_col] = 1000
        self.apple_location = [rand_row, rand_col]
        #Set Snake Head location
        snake_loc = False
        while snake_loc == False:
            snake_row = random.randint(0, (self.rows-1))
            snake_col = random.randint(0, (self.columns-1))
            if self.board_state[snake_row, snake_col] != 1000:
                self.board_state[snake_row, snake_col] = self.snake_max_len
                snake_loc = True
                self.snake_location = [snake_row, snake_col]
        #Set Snake Body Location
        snake_length = 1
        snake_val = self.snake_max_len
        moves = self.moves
        snake_built = False
        last_row = snake_row
        last_col = snake_col
        while snake_built == False:
            viable_space = []
            for move in moves:
                viable = True
                if (last_row + move[0]) < 0:
                    viable = False
                else:
                    if (last_row + move[0]) >= self.rows:
                        viable = False
                if (last_col + move[1]) < 0:
                    viable = False
                else:
                    if (last_col + move[1]) >= self.columns:
                        viable = False
                if viable == True:
                    if self.board_state[(last_row+move[0]), (last_col+move[1])] != 0:
                        viable = False
                    if viable == True:
                        viable_space.append(move)
            if len(viable_space) == 0:
                breakpoint()
            rand_move = viable_space[random.randint(0, (len(viable_space)-1))]
            last_row = last_row + rand_move[0]
            last_col = last_col + rand_move[1]
            snake_val -= 1
            self.board_state[last_row, last_col] = snake_val
            if snake_val <= (self.snake_max_len - self.snake_starting_length):
                snake_built = True
        self.snake_length = self.snake_starting_length
        return self.board_state
    
    def render(self, mode='human', close=False):
        pass
    
    def close(self):
        pass

In [3]:
def _policy_(board):
    #Set Game Perameters
    row_count = 10
    column_count = 10

    #Find Snakes Head
    find = np.where(board==100)
    moves = [(1, 0), (0, 1), (-1, 0), (0, -1)]
    
    #Find Apple Location
    find_apple = np.where(board==1000)

    #Search for viable local moves
    viable_moves = []
    for move in moves:
        inbounds = True
        if (find[0]+move[0]) < 0:
            inbounds = False
        elif (find[0]+move[0]) >= row_count:
            inbounds = False
        if (find[1]+move[1]) < 0:
            inbounds = False
        elif (find[1]+move[1]) >= column_count:
            inbounds = False
        if inbounds == True:
            value = board[(find[0]+move[0]), (find[1]+move[1])]
            if value == 0:
                viable_moves.append(move)
            elif value == 1000:
                viable_moves.append(move)
    #Find Move with shortest distance
    shortest_distance = 100
    best_move = []
    for move in viable_moves:
        row_dist = abs(find_apple[0] - (find[0] + move[0]))[0]
        col_dist = abs(find_apple[1] - (find[1] + move[1]))[0]
        total_distance = row_dist + col_dist
        if total_distance == shortest_distance:
            best_move.append(move)
        elif total_distance < shortest_distance:
            shortest_distance = total_distance
            best_move = [move]
    #Select move
    if len(best_move) == 0:
        deciding_move = random.randint(0, 3)
    else:
        move_int = random.randint(0, (len(best_move)-1))
        deciding_move = moves.index(best_move[move_int])
    return deciding_move

In [4]:
env = snake()
board = env.reset()

for _ in range(100):
    policy = _policy_(board)
    #policy = int(input()) # 0 is down, 1 is right, 2 is up, 3 is left3
    board, reward, done = env.step(policy)
    if done == True:
        print(f'Game Finished reward: {reward}')
        break
    if _ > 100:
        print('Time Exceeded')
        break

for f in env.videos:
    video = io.open(f[0], 'r+b').read()
    encoded = base64.b64encode(video)

    display.display(display.HTML(data="""
        <video alt="test" controls>
        <source src="data:video/mp4;base64,{0}" type="video/mp4" />
        </video>
        """.format(encoded.decode('ascii'))))

Game Finished reward: 1
